In [2]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
%load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [5]:
query2=f"""
SELECT
A.*, B.LapTime, B.Stint, B.PitOutTime, B.PitInTime,B.Sector1Time, B.Sector2Time, B.Sector3Time, B.Compound, B.TyreLife,B.FreshTyre, B.TrackStatus, B.Position
FROM
`perceptive-ivy-290216.f1_api.race_driver_ahead_2024` A
LEFT JOIN
`perceptive-ivy-290216.f1_api.race_lap_time` B
ON
A.Year=B.Year
AND
A.GP=B.GP
AND
A.Driver=B.Driver
AND
A.LapNumber=B.LapNumber
WHERE
A.GP="Australian Grand Prix"
# AND
# B.GP="Australian Grand Prix"
# AND
# A.Driver="SAI"
ORDER BY 1,2,3,4,5,6,7
"""
track2=pd.read_gbq(query2,project_id,dialect='standard')

In [6]:
track2.head()

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime,Distance,DriverAhead,DistanceToDriverAhead,LapNumber,Year,GP,Driver,LapTime,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Compound,TyreLife,FreshTyre,TrackStatus,Position
0,2024-03-24 04:03:12.649,3977,2,0,0,False,1,car,0 days 00:00:00.219000,0 days 00:58:29.451000,0.121667,,NaN,1,2024,Australian Grand Prix,ZHO,0 days 00:01:38.691000,1.0,0 days 00:58:37.885000,NaT,NaT,0 days 00:00:18.721000,0 days 00:00:36.608000,SOFT,1.0,True,1,19.0
1,2024-03-24 04:03:12.649,8812,0,1,21,False,1,car,0 days 00:00:00.219000,0 days 00:58:29.451000,0.000000,,NaN,1,2024,Australian Grand Prix,HAM,0 days 00:01:33.688000,1.0,NaT,NaT,NaT,0 days 00:00:18.674000,0 days 00:00:36.983000,SOFT,1.0,True,1,10.0
2,2024-03-24 04:03:12.649,9063,0,1,21,False,1,car,0 days 00:00:00.219000,0 days 00:58:29.451000,0.000000,,NaN,1,2024,Australian Grand Prix,RUS,0 days 00:01:31.910000,1.0,NaT,NaT,NaT,0 days 00:00:18.460000,0 days 00:00:36.821000,MEDIUM,1.0,True,1,6.0
3,2024-03-24 04:03:12.649,9342,0,1,23,True,1,car,0 days 00:00:00.219000,0 days 00:58:29.451000,0.000000,,NaN,1,2024,Australian Grand Prix,STR,0 days 00:01:32.835000,1.0,NaT,NaT,NaT,0 days 00:00:18.589000,0 days 00:00:37.079000,MEDIUM,2.0,False,1,8.0
4,2024-03-24 04:03:12.649,9435,0,1,21,False,1,car,0 days 00:00:00.219000,0 days 00:58:29.451000,0.000000,,NaN,1,2024,Australian Grand Prix,NOR,0 days 00:01:29.784000,1.0,NaT,NaT,NaT,0 days 00:00:18.490000,0 days 00:00:36.041000,MEDIUM,1.0,True,1,3.0


In [7]:
track2["Time"]=track2['Time'].str.split('days ').str[1]

track2["SessionTime"]=track2['SessionTime'].str.split('days ').str[1]
track2["SessionTime"]=pd.to_datetime(track2['SessionTime'],format='%H:%M:%S.%f')
track2["SessionTime"]=track2["SessionTime"].dt.strftime("%H:%M:%S.%f")

In [8]:
#Convert lapnumber to string for discrete chart coloring
track2["LapNumber"]=track2["LapNumber"].astype(str)

In [9]:
track2['Dist_Previous'] = track2.groupby(["LapNumber","Driver"])["Distance"].shift(1)
track2['Dist_Traveled_Incremental'] = track2['Distance']-track2["Dist_Previous"]
track2["Dist_Total_Race"] = track2.groupby(by=["Driver"])["Dist_Traveled_Incremental"].cumsum()
# track2["Dist_Diff_Sum"] = track2.groupby(by=["Driver"])["Dist_Diff"].cumsum()  ## Use when not looking at just 1 lap

# #Assign Rank for each entry point
track2["RK"] = track2.groupby(by=["LapNumber","Driver"])["Time"].rank(method="dense", ascending=True)
track2["RK"]= track2["RK"].astype(int)

In [10]:
track2.sort_values(by=["SessionTime","Driver"], inplace=True)

In [11]:
track2['SessionTime_Converted']= pd.to_timedelta(track2["SessionTime"])
track2.loc[:, "SessionTime (s)"] = track2["SessionTime_Converted"].dt.total_seconds()

In [12]:
track2[track2["Driver"]=="LEC"].tail()

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime,Distance,DriverAhead,DistanceToDriverAhead,LapNumber,Year,GP,Driver,LapTime,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Compound,TyreLife,FreshTyre,TrackStatus,Position,Dist_Previous,Dist_Traveled_Incremental,Dist_Total_Race,RK,SessionTime_Converted,SessionTime (s)
313934,2024-03-24 05:23:40.425,8821,167,5,0,False,1,car,00:01:48.437000,02:18:57.227000,5150.822778,,NaN,58,2024,Australian Grand Prix,LEC,0 days 00:01:49.995000,3.0,NaT,NaT,0 days 00:00:36.732000,0 days 00:00:29.762000,0 days 00:00:43.501000,HARD,24.0,True,2,2.0,5143.400556,7.422222,301999.552778,402,0 days 02:18:57.227000,8337.227
313948,2024-03-24 05:23:40.785,8515,162,5,0,False,1,car,00:01:48.797000,02:18:57.587000,5167.022778,,NaN,58,2024,Australian Grand Prix,LEC,0 days 00:01:49.995000,3.0,NaT,NaT,0 days 00:00:36.732000,0 days 00:00:29.762000,0 days 00:00:43.501000,HARD,24.0,True,2,2.0,5150.822778,16.200000,302015.752778,403,0 days 02:18:57.587000,8337.587
313962,2024-03-24 05:23:41.145,8352,160,5,1,False,1,car,00:01:49.157000,02:18:57.947000,5183.022778,,NaN,58,2024,Australian Grand Prix,LEC,0 days 00:01:49.995000,3.0,NaT,NaT,0 days 00:00:36.732000,0 days 00:00:29.762000,0 days 00:00:43.501000,HARD,24.0,True,2,2.0,5167.022778,16.000000,302031.752778,404,0 days 02:18:57.947000,8337.947
313977,2024-03-24 05:23:41.505,8212,157,5,1,False,1,car,00:01:49.517000,02:18:58.307000,5198.722778,,NaN,58,2024,Australian Grand Prix,LEC,0 days 00:01:49.995000,3.0,NaT,NaT,0 days 00:00:36.732000,0 days 00:00:29.762000,0 days 00:00:43.501000,HARD,24.0,True,2,2.0,5183.022778,15.700000,302047.452778,405,0 days 02:18:58.307000,8338.307
313994,2024-03-24 05:23:41.865,9362,156,4,1,False,1,car,00:01:49.877000,02:18:58.667000,5214.322778,,NaN,58,2024,Australian Grand Prix,LEC,0 days 00:01:49.995000,3.0,NaT,NaT,0 days 00:00:36.732000,0 days 00:00:29.762000,0 days 00:00:43.501000,HARD,24.0,True,2,2.0,5198.722778,15.600000,302063.052778,406,0 days 02:18:58.667000,8338.667


In [13]:
px.line(data_frame=track2[track2["LapNumber"]=='2'],
        x="SessionTime",
        y="Dist_Total_Race",
        color="Driver", hover_data=["Date","Speed","nGear","Throttle","Brake","DRS","Time","SessionTime","Distance","DriverAhead","DistanceToDriverAhead","LapNumber","Year","GP","Driver","Dist_Traveled_Incremental","Dist_Total_Race"],
        height=900, width=1800,
          #  template="plotly_dark",
        title="Speed Trace For GP",
        )

In [21]:
# animation=track2[(track2["LapNumber"].isin(['1','2','3','4','5']))&(track2["Driver"].isin(["VER","SAI"]))]
animation=track2[track2["LapNumber"].isin(['1','2','3','4','5'])]

In [22]:
animation=animation.sort_values(by=["Driver","SessionTime"])

In [ ]:
fig=px.scatter(
          animation,
          x="Dist_Total_Race",
          y="Position",
          animation_frame="SessionTime",
          animation_group="Driver",
          color="Driver",
          hover_data=["Position","Date","Speed","nGear","Throttle","Brake","DRS","Time","SessionTime","Distance","DriverAhead","DistanceToDriverAhead","LapNumber","Year","GP","Driver","Dist_Traveled_Incremental","Dist_Total_Race"],
           size="Position",
          text="Driver",
           range_x=[0,20000],
           range_y=[0,21],
           height=900,
           width=1500,
           )
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
fig.show()

In [ ]:
fig.show()

In [17]:
fig=px.scatter(
          track2,
          x="Driver",
          y="Position",
          animation_frame="LapNumber",
          animation_group="Driver",
          color="Driver",
          # hover_name="Driver",
          #  size="Position",
          #  range_x=[3600,3700],
           range_y=[0,21],
           height=900,
           width=1500,
           )
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 100
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 10
fig.show()

KeyboardInterrupt: 